In [1]:
import pandas as pd
import seaborn as sns
import itertools
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_1bed_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_1_bed_flat_monetary_2013-2021.csv')
bed_2_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_flat_monetary_2013-2021.csv')
bed_2_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_house_monetary_2013-2021.csv')
bed_3_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_flat_monetary_2013-2021.csv')
bed_3_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_house_monetary_2013-2021.csv')
bed_4_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_4_house_monetary_2013-2021.csv')

In [103]:
df_1bed_flat

,Count_Mar2013,Median_Mar2013,Count_Jun2013,Median_Jun2013,Count_Sep2013,Median_Sep2013,Count_Dec2013,Median_Dec2013,Count_Mar2014,Median_Mar2014,...,Victimisation_Rate_2021,Offence_Count_2013,Offence_Count_2014,Offence_Count_2015,Offence_Count_2016,Offence_Count_2017,Offence_Count_2018,Offence_Count_2019,Offence_Count_2020,Offence_Count_2021
0,298,$330,300,$330,320,$320,325,$320,328,$320,...,"5,265.30","11,054","11,364","11,915","12,733","13,427","12,549","12,778","12,965","13,408"
1,180,$305,178,$300,171,$300,173,$300,168,$300,...,"4,398.30","8,603","9,444","9,958","10,204","11,484","10,191","10,501","12,090","11,635"
2,70,$310,70,$310,60,$310,58,$310,56,$313,...,"6,229.20","33,752",890,"33,707","35,452","37,315","36,659","35,626","36,359","35,712"
3,"2,280",$320,"2,293",$320,"2,277",$325,"2,231",$325,"2,367",$329,...,"6,229.20","33,752",890,"33,707","35,452","37,315","36,659","35,626","36,359","35,712"
4,"3,322",$380,"3,522",$380,"3,700",$380,"3,904",$380,"4,045",$375,...,"5,265.30","11,054","11,364","11,915","12,733","13,427","12,549","12,778","12,965","13,408"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,14,$250,15,$250,18,$240,17,$230,15,$250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,72,$140,84,$140,78,$140,77,$140,82,$148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,-,-,-,-,-,-,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,80,$160,87,$160,91,$160,83,$170,87,$175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_1bed_flat_no_covid =pd.read_csv('../data/curated/Rental_Price_historical/joined_1_bed_flat_monetary_2013-2019.csv')
bed_2_flat_no_covid=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_flat_monetary_2013-2019.csv')
bed_2_house_no_covid=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_house_monetary_2013-2019.csv')
bed_3_flat_no_covid=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_flat_monetary_2013-2019.csv')
bed_3_house_no_covid=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_house_monetary_2013-2019.csv')
bed_4_house_no_covid=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_4_house_monetary_2013-2019.csv')

# Prediction
The features are:
1. Median_age_persons
3. Median_tot_prsnl_inc_weekly
4. Average_household_size
5. Population
6. Alleged Offender Incident Rate
7. Victimization Rate
8. Offender Count

In [4]:
# Function returns forecasted values of given years for each suburb
def get_forecast_all_suburbs(raw_df, yrs, PRED_COL):
    # cols_imputed = {}
    forecasted_data = {}
    for index, row in raw_df.iterrows():
        X_df = make_2D(row, yrs, PRED_COL)


        forecasted_data[row['Suburb']] = forecast(X_df)


    return forecasted_data


# Function returns 2D version of a given row of time series dataset
def make_2D(suburb_row, yrs, PRED_COL):
    cols_imputed = {}
    X_df = {}
    for feature in PRED_COL:
        f = []
        for t in suburb_row.iteritems():
            if feature in t[0]:
                f.append(t[1])
        X_df[feature] = f

    X_df = pd.DataFrame.from_dict(X_df)
    X_df.loc[:,'year'] = yrs
    X_df.rename(columns = {"Median_Mar":"Median_rental_price"}, inplace = True)
    
    for col in X_df:
        year = []
        data = []
        if col != 'year':
            for i in range(0,len(yrs)):
                if (X_df.loc[i,col] != -1):
                    data.append(X_df.loc[i,col])
                    year.append(yrs[i]) 
            if (len(year) == 0):
                continue   
            df = pd.DataFrame()     
            df["year"]=year
            df["data"]=data 
            reg = LinearRegression().fit(df[["year"]], df["data"])
            
            arr = []
            for yr in yrs:
                arr.extend(reg.predict(np.array([[yr]])))
    #                     print(arr)
        cols_imputed[col] = arr


    for i in range(0,len(yrs)):
        for col in cols_imputed:
            if (X_df.loc[i,col] == -1):
                X_df.loc[i,col] = cols_imputed[col][i]

    return X_df


# Function that makes predictions using SVR and return forecasted values.
def forecast(X_df):
    future_yrs = [2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030]
    reg_forecast = pd.DataFrame()
    for col in X_df.iloc[:, 1:-1]:
        l = []
        reg = LinearRegression().fit(X_df['year'].values.reshape((-1,1)), X_df[col].values.reshape((-1,1)))
        for yr in future_yrs:
            l.extend(reg.predict(np.array([[yr]])))
            row = [np.round(x[0], 1) for x in l]
    #         print(l)
        reg_forecast[col] = row
    reg_forecast['year'] = future_yrs
    regr = make_pipeline(StandardScaler(), SVR(kernel='poly', C=3, epsilon=0.2))

    regr.fit(X_df.iloc[:,[1,2,3,7,8,9,10,11]], X_df['Median_rental_price'])
    future_price = regr.predict(reg_forecast.iloc[:,[0,1,2,6,7,8,9,10]])


#     print(reg_forecast.iloc[:,[0,1,2,6]])
    # Median_age_persons, Median_mortgage_repay_monthly, Median_tot_prsnl_inc_weekly, Average_household_size
    
    reg_forecast["Median_rental_price"] = future_price
    # reg = LinearRegression().fit(X_df.iloc[:,[1,2,3,7]], X_df['Median_rental_price'])

    # future_price = reg.predict(reg_forecast.iloc[:,[0,1,2,6]])
    # reg_forecast["Median_rental_price"] = future_price
    return reg_forecast

#### Prediction

In [145]:
bed_4_house.nunique()

Count_Mar2013         103
Median_Mar2013         69
Count_Jun2013         102
Median_Jun2013         79
Count_Sep2013         101
                     ... 
Offence_Count_2017     29
Offence_Count_2018     29
Offence_Count_2019     29
Offence_Count_2020     29
Offence_Count_2021     29
Length: 168, dtype: int64

In [181]:
bed_4_house

,Count_Mar2013,Median_Mar2013,Count_Jun2013,Median_Jun2013,Count_Sep2013,Median_Sep2013,Count_Dec2013,Median_Dec2013,Count_Mar2014,Median_Mar2014,...,Victimisation_Rate_2021,Offence_Count_2013,Offence_Count_2014,Offence_Count_2015,Offence_Count_2016,Offence_Count_2017,Offence_Count_2018,Offence_Count_2019,Offence_Count_2020,Offence_Count_2021
0,35,"$1,100",40,"$1,125",32,"$1,150",38,"$1,125",41,"$1,050",...,"5,265.30","11,054","11,364","11,915","12,733","13,427","12,549","12,778","12,965","13,408"
1,15,"$1,175",14,"$1,138",12,"$1,200",13,"$1,300",11,"$1,200",...,"4,398.30","8,603","9,444","9,958","10,204","11,484","10,191","10,501","12,090","11,635"
2,11,$900,14,$888,13,$880,12,$850,17,$850,...,"6,229.20","33,752",890,"33,707","35,452","37,315","36,659","35,626","36,359","35,712"
3,37,$800,28,$819,34,$820,28,$850,31,$850,...,"6,229.20","33,752",890,"33,707","35,452","37,315","36,659","35,626","36,359","35,712"
4,-,-,-,-,-,-,-,-,-,-,...,"5,265.30","11,054","11,364","11,915","12,733","13,427","12,549","12,778","12,965","13,408"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,152,$450,152,$450,150,$450,130,$460,145,$460,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,117,$350,133,$350,147,$350,155,$350,154,$350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,99,$335,102,$330,106,$340,102,$340,108,$340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,183,$380,183,$390,185,$395,179,$390,181,$380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Function replaces all missing values and typecast int to appropriate columns.
def impute_missing(df, impute_value, pred_cols):
    for col in df:
        df[col] = df[col].replace("-",impute_value)
        for pred_col in pred_cols:
            if pred_col in col:
                df[col] = df[col].replace({'\$':''}, regex = True)
                df[col] = df[col].replace(",","", regex=True)
                df[col] = df[col].astype(float)
    return df

In [11]:
pred_cols = ['Median_Mar', 'Median_age_persons', 'Median_mortgage_repay_monthly', 'Median_tot_prsnl_inc_weekly', 
             'Median_rent_weekly', 'Median_tot_fam_inc_weekly','Average_num_psns_per_bedroom', 
             'Average_household_size', 'Pop', 'Alleged_Offender_Incident_Rate', 'Victimisation_Rate', 'Offence_Count']

In [7]:
df_1bed_flat = impute_missing(df_1bed_flat, "-1", pred_cols)
bed_2_flat = impute_missing(bed_2_flat, "-1", pred_cols)
bed_2_house = impute_missing(bed_2_house, "-1", pred_cols)
bed_3_flat = impute_missing(bed_3_flat, "-1", pred_cols)
bed_3_house = impute_missing(bed_3_house, "-1", pred_cols)
bed_4_house = impute_missing(bed_4_house, "-1", pred_cols)

In [185]:
df_1bed_flat

,Count_Mar2013,Median_Mar2013,Count_Jun2013,Median_Jun2013,Count_Sep2013,Median_Sep2013,Count_Dec2013,Median_Dec2013,Count_Mar2014,Median_Mar2014,...,Victimisation_Rate_2021,Offence_Count_2013,Offence_Count_2014,Offence_Count_2015,Offence_Count_2016,Offence_Count_2017,Offence_Count_2018,Offence_Count_2019,Offence_Count_2020,Offence_Count_2021
0,298,330.0,300,$330,320,$320,325,$320,328,320.0,...,5265.3,11054.0,11364.0,11915.0,12733.0,13427.0,12549.0,12778.0,12965.0,13408.0
1,180,305.0,178,$300,171,$300,173,$300,168,300.0,...,4398.3,8603.0,9444.0,9958.0,10204.0,11484.0,10191.0,10501.0,12090.0,11635.0
2,70,310.0,70,$310,60,$310,58,$310,56,313.0,...,6229.2,33752.0,890.0,33707.0,35452.0,37315.0,36659.0,35626.0,36359.0,35712.0
3,"2,280",320.0,"2,293",$320,"2,277",$325,"2,231",$325,"2,367",329.0,...,6229.2,33752.0,890.0,33707.0,35452.0,37315.0,36659.0,35626.0,36359.0,35712.0
4,"3,322",380.0,"3,522",$380,"3,700",$380,"3,904",$380,"4,045",375.0,...,5265.3,11054.0,11364.0,11915.0,12733.0,13427.0,12549.0,12778.0,12965.0,13408.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,14,250.0,15,$250,18,$240,17,$230,15,250.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,72,140.0,84,$140,78,$140,77,$140,82,148.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,80,160.0,87,$160,91,$160,83,$170,87,175.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [186]:
df_1bed_flat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Columns: 168 entries, Count_Mar2013 to Offence_Count_2021
dtypes: float64(108), int64(1), object(59)
memory usage: 190.4+ KB


In [6]:
yrs = [2013,2014,2015,2016,2017,2018,2019, 2020, 2021]



df_1bed_flat_w_pop_cleaned = df_1bed_flat.dropna()
bed_2_flat_w_pop_cleaned = bed_2_flat.dropna()
bed_2_house_w_pop_cleaned = bed_2_house.dropna()
bed_3_flat_w_pop_cleaned = bed_3_flat.dropna()
bed_3_house_w_pop_cleaned = bed_3_house.dropna()
bed_4_house_w_pop_cleaned = bed_4_house.dropna()

In [189]:
bed_2_flat_w_pop_cleaned

,Count_Mar2013,Median_Mar2013,Count_Jun2013,Median_Jun2013,Count_Sep2013,Median_Sep2013,Count_Dec2013,Median_Dec2013,Count_Mar2014,Median_Mar2014,...,Victimisation_Rate_2021,Offence_Count_2013,Offence_Count_2014,Offence_Count_2015,Offence_Count_2016,Offence_Count_2017,Offence_Count_2018,Offence_Count_2019,Offence_Count_2020,Offence_Count_2021
0,255,450.0,266,$450,256,$450,268,$450,264,450.0,...,5265.3,11054.0,11364.0,11915.0,12733.0,13427.0,12549.0,12778.0,12965.0,13408.0
1,264,395.0,276,$395,287,$395,305,$395,328,395.0,...,4398.3,8603.0,9444.0,9958.0,10204.0,11484.0,10191.0,10501.0,12090.0,11635.0
2,99,415.0,103,$420,105,$415,105,$410,105,400.0,...,6229.2,33752.0,890.0,33707.0,35452.0,37315.0,36659.0,35626.0,36359.0,35712.0
3,951,455.0,947,$455,969,$455,965,$456,"1,045",460.0,...,6229.2,33752.0,890.0,33707.0,35452.0,37315.0,36659.0,35626.0,36359.0,35712.0
4,"2,900",520.0,"2,973",$520,"3,050",$520,"3,097",$520,"3,369",520.0,...,5265.3,11054.0,11364.0,11915.0,12733.0,13427.0,12549.0,12778.0,12965.0,13408.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,195,250.0,209,$250,206,$250,210,$250,198,250.0,...,2243.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0,11317.0,10774.0
106,744,260.0,763,$260,741,$260,765,$260,755,265.0,...,3319.1,13223.0,890.0,14402.0,15288.0,16647.0,14308.0,14787.0,14942.0,14157.0
107,220,260.0,237,$260,235,$260,239,$262,226,260.0,...,2243.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0,11317.0,10774.0
108,275,300.0,261,$300,262,$300,258,$310,253,310.0,...,2243.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0,11317.0,10774.0


In [58]:
df_1bed_flat_w_pop_cleaned.to_csv('../data/curated/Rental_Price_historical/joined_1_bed_flat_monetary_2013-2021_cleaned.csv', index = False)
bed_2_flat_w_pop_cleaned.to_csv('../data/curated/Rental_Price_historical/joined_bed_2_flat_monetary_2013-2021_cleaned.csv', index = False)
bed_2_house_w_pop_cleaned.to_csv('../data/curated/Rental_Price_historical/joined_bed_2_house_monetary_2013-2021_cleaned.csv', index = False)
bed_3_flat_w_pop_cleaned.to_csv('../data/curated/Rental_Price_historical/joined_bed_3_flat_monetary_2013-2021_cleaned.csv', index = False)
bed_3_house_w_pop_cleaned.to_csv('../data/curated/Rental_Price_historical/joined_bed_3_house_monetary_2013-2021_cleaned.csv', index = False)
bed_4_house_w_pop_cleaned.to_csv('../data/curated/Rental_Price_historical/joined_bed_4_house_monetary_2013-2021_cleaned.csv', index = False)

In [7]:
df_1bed_flat_w_pop_cleaned = pd.read_csv('../data/curated/Rental_Price_historical/joined_1_bed_flat_monetary_2013-2021_cleaned.csv')
bed_2_flat_w_pop_cleaned = pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_flat_monetary_2013-2021_cleaned.csv')
bed_2_house_w_pop_cleaned = pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_house_monetary_2013-2021_cleaned.csv')
bed_3_flat_w_pop_cleaned = pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_flat_monetary_2013-2021_cleaned.csv')
bed_3_house_w_pop_cleaned = pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_house_monetary_2013-2021_cleaned.csv')
bed_4_house_w_pop_cleaned = pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_4_house_monetary_2013-2021_cleaned.csv')

In [60]:
df_1bed_flat_w_pop_cleaned

,Count_Mar2013,Median_Mar2013,Count_Jun2013,Median_Jun2013,Count_Sep2013,Median_Sep2013,Count_Dec2013,Median_Dec2013,Count_Mar2014,Median_Mar2014,...,Victimisation_Rate_2021,Offence_Count_2013,Offence_Count_2014,Offence_Count_2015,Offence_Count_2016,Offence_Count_2017,Offence_Count_2018,Offence_Count_2019,Offence_Count_2020,Offence_Count_2021
0,298,330.0,300,$330,320,$320,325,$320,328,320.0,...,5265.3,11054.0,11364.0,11915.0,12733.0,13427.0,12549.0,12778.0,12965.0,13408.0
1,180,305.0,178,$300,171,$300,173,$300,168,300.0,...,4398.3,8603.0,9444.0,9958.0,10204.0,11484.0,10191.0,10501.0,12090.0,11635.0
2,70,310.0,70,$310,60,$310,58,$310,56,313.0,...,6229.2,33752.0,890.0,33707.0,35452.0,37315.0,36659.0,35626.0,36359.0,35712.0
3,"2,280",320.0,"2,293",$320,"2,277",$325,"2,231",$325,"2,367",329.0,...,6229.2,33752.0,890.0,33707.0,35452.0,37315.0,36659.0,35626.0,36359.0,35712.0
4,"3,322",380.0,"3,522",$380,"3,700",$380,"3,904",$380,"4,045",375.0,...,5265.3,11054.0,11364.0,11915.0,12733.0,13427.0,12549.0,12778.0,12965.0,13408.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,40,198.0,41,$220,34,$220,31,$220,23,230.0,...,2243.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0,11317.0,10774.0
90,158,190.0,167,$200,199,$200,226,$200,248,210.0,...,3319.1,13223.0,890.0,14402.0,15288.0,16647.0,14308.0,14787.0,14942.0,14157.0
91,22,240.0,20,$240,17,$240,24,$205,21,194.0,...,2243.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0,11317.0,10774.0
92,24,218.0,27,$215,33,$215,31,$220,27,220.0,...,2243.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0,11317.0,10774.0


In [64]:
forecast_bed_1_flat_covid = get_forecast_all_suburbs(df_1bed_flat_w_pop_cleaned, yrs, pred_cols)
forecast_bed_2_flat_covid = get_forecast_all_suburbs(bed_2_flat_w_pop_cleaned, yrs, pred_cols)
forecast_bed_2_house_covid = get_forecast_all_suburbs(bed_2_house_w_pop_cleaned, yrs, pred_cols)
forecast_bed_3_flat_covid = get_forecast_all_suburbs(bed_3_flat_w_pop_cleaned, yrs, pred_cols)
forecast_bed_3_house_covid = get_forecast_all_suburbs(bed_3_house_w_pop_cleaned, yrs, pred_cols)
forecast_bed_4_house_covid = get_forecast_all_suburbs(bed_4_house_w_pop_cleaned, yrs, pred_cols)

In [203]:
import pickle
filename = 'forecast_bed_2_house_covid'
outfile = open(filename,'wb')
pickle.dump(forecast_bed_2_house_covid,outfile)
outfile.close()

In [211]:
with open('../data/curated/forecast_bed_1_flat_covid', 'rb') as f:
    forecast_bed_1_flat_covid = pickle.load(f)
with open('../data/curated/forecast_bed_2_flat_covid', 'rb') as f:
    forecast_bed_2_flat_covid = pickle.load(f)
with open('../data/curated/forecast_bed_2_house_covid', 'rb') as f:
    forecast_bed_2_house_covid = pickle.load(f)

In [216]:
bed_2_flat_w_pop_cleaned.loc[1,'Suburb']

'Armadale North, Armadale'

In [215]:
forecast_bed_1_flat_covid[bed_2_flat_w_pop_cleaned.loc[1,'Suburb']]

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop,Alleged_Offender_Incident_Rate,Victimisation_Rate,Offence_Count,year,Median_rental_price
0,37.8,2427.3,1289.9,412.7,3216.5,0.9,2.0,10480.2,2763.5,4585.3,11513.6,2020,328.495861
1,38.0,2427.4,1333.4,419.9,3306.3,0.9,2.0,10573.7,2822.9,4643.3,11865.9,2021,336.677672
2,38.2,2427.5,1376.8,427.2,3396.1,0.9,2.0,10667.2,2882.3,4701.3,12218.2,2022,348.805872
3,38.4,2427.6,1420.3,434.4,3485.9,0.8,2.0,10760.7,2941.7,4759.3,12570.6,2023,365.677220
4,38.6,2427.7,1463.8,441.7,3575.7,0.8,2.0,10854.2,3001.1,4817.3,12922.9,2024,388.067073
5,38.8,2427.8,1507.3,448.9,3665.5,0.8,2.0,10947.7,3060.5,4875.2,13275.2,2025,416.755303
6,39.0,2427.9,1550.8,456.2,3755.3,0.8,2.0,11041.2,3119.9,4933.2,13627.5,2026,452.531082
7,39.2,2428.0,1594.3,463.4,3845.1,0.8,1.9,11134.7,3179.3,4991.2,13979.8,2027,600.609407
8,39.4,2428.1,1637.7,470.7,3934.9,0.8,1.9,11228.2,3238.7,5049.2,14332.1,2028,670.634358
9,39.6,2428.2,1681.2,477.9,4024.7,0.7,1.9,11321.7,3298.1,5107.2,14684.5,2029,751.539274


In [213]:
forecast_bed_2_flat_covid[bed_2_flat_w_pop_cleaned.loc[0,'Suburb']]

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop,Alleged_Offender_Incident_Rate,Victimisation_Rate,Offence_Count,year,Median_rental_price
0,36.3,1995.7,1181.3,379.5,2690.0,1.0,1.7,3905.9,3105.2,5593.1,13253.9,2020,409.881249
1,36.8,1985.7,1209.8,383.3,2750.0,1.0,1.7,3917.0,3120.7,5565.0,13516.6,2021,420.707281
2,37.2,1975.6,1238.4,387.0,2810.0,1.0,1.7,3928.1,3136.2,5536.9,13779.3,2022,436.580748
3,37.6,1965.5,1266.9,390.8,2869.9,1.0,1.7,3939.2,3151.8,5508.8,14042.0,2023,458.930775
4,38.1,1955.4,1295.4,394.6,2929.9,0.9,1.7,3950.3,3167.3,5480.7,14304.7,2024,490.391740
5,38.5,1945.3,1324.0,398.4,2989.9,0.9,1.7,3961.4,3182.8,5452.7,14567.4,2025,529.498973
6,38.9,1935.2,1352.5,402.2,3049.8,0.9,1.7,3972.5,3198.3,5424.6,14830.0,2026,578.455647
7,39.4,1925.2,1381.0,406.0,3109.8,0.9,1.6,3983.6,3213.9,5396.5,15092.7,2027,738.763641
8,39.8,1915.1,1409.6,409.7,3169.8,0.9,1.6,3994.7,3229.4,5368.4,15355.4,2028,828.797801
9,40.2,1905.0,1438.1,413.5,3229.7,0.8,1.6,4005.8,3244.9,5340.3,15618.1,2029,933.476136


In [214]:
forecast_bed_2_house_covid[bed_2_flat_w_pop_cleaned.loc[0,'Suburb']]

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop,Alleged_Offender_Incident_Rate,Victimisation_Rate,Offence_Count,year,Median_rental_price
0,36.3,1995.7,1181.3,379.5,2690.0,1.0,1.7,3905.9,3105.2,5593.1,13253.9,2020,569.779527
1,36.8,1985.7,1209.8,383.3,2750.0,1.0,1.7,3917.0,3120.7,5565.0,13516.6,2021,579.122209
2,37.2,1975.6,1238.4,387.0,2810.0,1.0,1.7,3928.1,3136.2,5536.9,13779.3,2022,592.887057
3,37.6,1965.5,1266.9,390.8,2869.9,1.0,1.7,3939.2,3151.8,5508.8,14042.0,2023,612.319272
4,38.1,1955.4,1295.4,394.6,2929.9,0.9,1.7,3950.3,3167.3,5480.7,14304.7,2024,639.693392
5,38.5,1945.3,1324.0,398.4,2989.9,0.9,1.7,3961.4,3182.8,5452.7,14567.4,2025,673.804376
6,38.9,1935.2,1352.5,402.2,3049.8,0.9,1.7,3972.5,3198.3,5424.6,14830.0,2026,716.557658
7,39.4,1925.2,1381.0,406.0,3109.8,0.9,1.6,3983.6,3213.9,5396.5,15092.7,2027,851.550577
8,39.8,1915.1,1409.6,409.7,3169.8,0.9,1.6,3994.7,3229.4,5368.4,15355.4,2028,929.453779
9,40.2,1905.0,1438.1,413.5,3229.7,0.8,1.6,4005.8,3244.9,5340.3,15618.1,2029,1020.156846


In [222]:
bed_2_house_w_pop_cleaned.loc[:,['Suburb', 'Postcode']].to_csv('../data/curated/sub_4_pred.csv')

In [158]:
bed_4_house.loc[6,'Suburb']

'Docklands'

In [159]:
forecast_bed_2_house_covid[bed_4_house.loc[6,'Suburb']]

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop,Alleged_Offender_Incident_Rate,Victimisation_Rate,Offence_Count,year,Median_rental_price
0,31.5,2017.0,1127.6,432.2,2402.0,1.1,1.9,12208.5,6753.8,7059.0,37683.7,2020,-1.0
1,31.6,1944.8,1144.8,421.7,2413.1,1.1,1.8,12713.1,6373.3,6736.5,39671.9,2021,-1.0
2,31.8,1872.5,1162.0,411.2,2424.3,1.1,1.8,13217.7,5992.8,6414.1,41660.1,2022,-1.0
3,31.9,1800.3,1179.3,400.7,2435.4,1.1,1.8,13722.3,5612.3,6091.6,43648.3,2023,-1.0
4,32.1,1728.1,1196.5,390.2,2446.5,1.1,1.8,14226.9,5231.8,5769.1,45636.5,2024,-1.0
5,32.2,1655.8,1213.7,379.7,2457.7,1.1,1.8,14731.5,4851.3,5446.6,47624.7,2025,-1.0
6,32.4,1583.6,1231.0,369.2,2468.8,1.1,1.8,15236.1,4470.8,5124.2,49612.9,2026,-1.0
7,32.5,1511.4,1248.2,358.7,2479.9,1.0,1.7,15740.7,4090.3,4801.7,51601.1,2027,-1.0
8,32.7,1439.1,1265.4,348.2,2491.1,1.0,1.7,16245.3,3709.8,4479.2,53589.3,2028,-1.0
9,32.8,1366.9,1282.7,337.7,2502.2,1.0,1.7,16749.9,3329.2,4156.7,55577.5,2029,-1.0


In [198]:
len(forecast_bed_2_house_covid)

90

In [117]:
subs = pd.read_csv('../data/curated/sub_4_pred.csv')
subs

,Suburb,Postcode
0,"St Kilda, St Kilda South, St Kilda West",3182
1,"Armadale North, Armadale",3143
2,"Carlton South, Carlton",3053
3,"Melbourne University, Parkville",3052
4,"St Kilda, St Kilda South, St Kilda West",3182
...,...,...
89,"Dromana, Arthurs Seat, Safety Beach",3936
90,"Karingal Centre, Karingal, Frankston Heights, ...",3199
91,"Tuerong, Hastings",3915
92,Mornington,3931


In [134]:
# Function to get average median price of forecast values/
def get_avg_median_price_2022_2027(forecast):
    mean = []
    for sub in subs['Suburb']:
        print(sub)
        curr_df = forecast[sub].iloc[2:8,:]
        print(curr_df)
        mean.append(curr_df['Median_rental_price'].mean())
        
    return mean

In [135]:
mean_bed_1_flat = get_avg_median_price_2022_2027(forecast_bed_1_flat_covid)
mean_bed_2_flat = get_avg_median_price_2022_2027(forecast_bed_2_flat_covid)
mean_bed_2_house = get_avg_median_price_2022_2027(forecast_bed_2_house_covid)

St Kilda, St Kilda South, St Kilda West
   Median_age_persons  Median_mortgage_repay_monthly  \
2                37.2                         1975.6   
3                37.6                         1965.5   
4                38.1                         1955.4   
5                38.5                         1945.3   
6                38.9                         1935.2   
7                39.4                         1925.2   

   Median_tot_prsnl_inc_weekly  Median_rent_weekly  Median_tot_fam_inc_weekly  \
2                       1238.4               387.0                     2810.0   
3                       1266.9               390.8                     2869.9   
4                       1295.4               394.6                     2929.9   
5                       1324.0               398.4                     2989.9   
6                       1352.5               402.2                     3049.8   
7                       1381.0               406.0                     3109.8   


   Median_age_persons  Median_mortgage_repay_monthly  \
2                41.2                         2501.2   
3                41.4                         2539.8   
4                41.6                         2578.3   
5                41.8                         2616.9   
6                42.0                         2655.4   
7                42.2                         2694.0   

   Median_tot_prsnl_inc_weekly  Median_rent_weekly  Median_tot_fam_inc_weekly  \
2                       1133.3               442.4                     3183.0   
3                       1170.2               453.1                     3273.4   
4                       1207.1               463.8                     3363.8   
5                       1244.0               474.5                     3454.1   
6                       1280.9               485.2                     3544.5   
7                       1317.8               495.9                     3634.9   

   Average_num_psns_per_bedroom  Averag

In [161]:
pd.DataFrame(mean_bed_1_flat).to_csv('../data/curated/mean_bed_1_flat_price_suburb_2022_2027.csv')
pd.DataFrame(mean_bed_2_flat).to_csv('../data/curated/mean_bed_2_flat_price_suburb_2022_2027.csv')
pd.DataFrame(mean_bed_2_house).to_csv('../data/curated/mean_bed_2_house_price_suburb_2022_2027.csv')

name of suburb and postcode was added in excel afterwards.

### Sorting median predicted price 2022 - 2027

In [171]:
mean_bed_1_flat = pd.read_csv('../data/curated/mean_bed_1_flat_price_suburb_2022_2027.csv')
mean_bed_2_flat = pd.read_csv('../data/curated/mean_bed_2_flat_price_suburb_2022_2027.csv')
mean_bed_2_house = pd.read_csv('../data/curated/mean_bed_2_house_price_suburb_2022_2027.csv')

In [173]:
mean_bed_1_flat.sort_values(['Mean_Median_Price'],
              ascending = True).to_csv('../data/curated/mean_bed_1_flat_price_suburb_2022_2027_sorted.csv')

In [174]:
mean_bed_2_flat.sort_values(['Mean_Median_Price'],
              ascending = True).to_csv('../data/curated/mean_bed_2_flat_price_suburb_2022_2027_sorted.csv')

In [175]:
mean_bed_2_house.sort_values(['Mean_Median_Price'],
              ascending = True).to_csv('../data/curated/mean_bed_2_house_price_suburb_2022_2027_sorted.csv')

In [255]:
IMPUTED_VALUE = -1
# Function to get average growth rate of Forecasted values
def get_avg_growth_rate(input_data):
    data = []
    for suburb in input_data.keys():
        rent = input_data[suburb]["Median_rental_price"]
        growth_rate = 0
        if rent[0] == IMPUTED_VALUE:
            continue
        for i in range(0, len(rent)-1):
            growth_rate += ((rent[i+1]-rent[i])/rent[i])*100
        avg_growth_rate = growth_rate/(len(rent)-1)
        data.append([suburb, avg_growth_rate])
    growth_rate_df = pd.DataFrame(data, columns = ["Suburb", "Average_Growth_Rate_%"])
    growth_rate_df = growth_rate_df.sort_values("Average_Growth_Rate_%", ascending = False)
    return growth_rate_df

In [262]:
forecast_bed_1_flat_covid['Rowville']

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop,Alleged_Offender_Incident_Rate,Victimisation_Rate,Offence_Count,year,Median_rental_price
0,40.4,1994.3,808.6,432.0,2330.9,0.8,2.9,34889.7,2226.9,2880.8,12145.4,2020,236.339064
1,40.9,2022.4,830.3,441.1,2394.2,0.8,2.9,34869.6,2314.7,2912.8,12964.2,2021,263.483113
2,41.4,2050.5,852.1,450.2,2457.6,0.8,2.9,34849.6,2402.5,2944.9,13782.9,2022,304.593675
3,41.9,2078.6,873.8,459.3,2520.9,0.7,2.9,34829.5,2490.3,2976.9,14601.7,2023,362.564153
4,42.4,2106.7,895.5,468.4,2584.2,0.7,2.8,34809.4,2578.1,3009.0,15420.5,2024,539.343183
5,42.9,2134.7,917.2,477.5,2647.5,0.7,2.8,34789.4,2665.9,3041.0,16239.2,2025,664.653877
6,43.5,2162.8,938.9,486.5,2710.8,0.7,2.8,34769.3,2753.7,3073.1,17058.0,2026,825.048797
7,44.0,2190.9,960.6,495.6,2774.1,0.7,2.8,34749.2,2841.5,3105.2,17876.8,2027,1011.484186
8,44.5,2219.0,982.4,504.7,2837.5,0.7,2.8,34729.2,2929.3,3137.2,18695.5,2028,1232.236991
9,45.0,2247.1,1004.1,513.8,2900.8,0.6,2.7,34709.1,3017.1,3169.3,19514.3,2029,1784.571500


In [256]:
growth_bed_1_flat = get_avg_growth_rate(forecast_bed_1_flat_covid)

In [257]:
growth_bed_1_flat

,Suburb,Average_Growth_Rate_%
76,Rowville,25.056144
74,"Upper Ferntree Gully, Lysterfield South, Mount...",18.025001
78,"Hoddles Creek, Wandin North, Beenak, Launching...",17.857746
55,"Tullamarine, Gowanbrae, Gladstone Park",17.689739
71,"Bayswater North, Bayswater",17.393221
...,...,...
27,"Cotham, Kew",3.401474
5,Docklands,1.848373
18,Bulleen,1.700684
13,"South Wharf, Southbank",-1.699726


In [258]:
growth_bed_2_flat = get_avg_growth_rate(forecast_bed_2_flat_covid)

In [259]:
growth_bed_2_flat

,Suburb,Average_Growth_Rate_%
74,"Upper Ferntree Gully, Lysterfield South, Mount...",18.933506
48,"Truganina, Tarneit, Hoppers Crossing",16.751796
44,"Spotswood, South Kingsville, Newport",16.438949
71,"Bayswater North, Bayswater",16.148808
40,"Seaholme, Altona",15.967861
...,...,...
11,"Richmond, Richmond North, Burnley North, Burnl...",4.835529
19,"Bennettswood, Surrey Hills South, Burwood",4.148467
4,"Collingwood, Collingwood North",3.429671
12,South Yarra,0.971094


In [260]:
growth_bed_2_house = get_avg_growth_rate(forecast_bed_2_house_covid)

In [261]:
growth_bed_2_house

,Suburb,Average_Growth_Rate_%
56,"Wildwood, Sunbury",16.940671
69,"Bayswater North, Bayswater",16.531204
84,Mornington,16.358594
72,"Upper Ferntree Gully, Lysterfield South, Mount...",15.788246
43,"St Albans, Kings Park, Kealba, Albanvale",15.467708
...,...,...
1,"Armadale North, Armadale",4.998379
6,"Brighton Road, Elwood",4.872686
25,"Cotham, Kew",4.295698
13,"Deepdene, Balwyn East, Balwyn, Stradbroke Park",3.762426


In [263]:
growth_bed_1_flat.to_csv('../data/curated/growth_bed_1_flat.csv')
growth_bed_2_flat.to_csv('../data/curated/growth_bed_2_flat.csv')
growth_bed_2_house.to_csv('../data/curated/growth_bed_2_house.csv')

In [225]:
forecast_bed_1_flat_covid['St Kilda, St Kilda South, St Kilda West']

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop,Alleged_Offender_Incident_Rate,Victimisation_Rate,Offence_Count,year,Median_rental_price
0,36.3,1995.7,1181.3,379.5,2690.0,1.0,1.7,3905.9,3105.2,5593.1,13253.9,2020,306.239351
1,36.8,1985.7,1209.8,383.3,2750.0,1.0,1.7,3917.0,3120.7,5565.0,13516.6,2021,311.532715
2,37.2,1975.6,1238.4,387.0,2810.0,1.0,1.7,3928.1,3136.2,5536.9,13779.3,2022,319.126491
3,37.6,1965.5,1266.9,390.8,2869.9,1.0,1.7,3939.2,3151.8,5508.8,14042.0,2023,329.700678
4,38.1,1955.4,1295.4,394.6,2929.9,0.9,1.7,3950.3,3167.3,5480.7,14304.7,2024,344.573531
5,38.5,1945.3,1324.0,398.4,2989.9,0.9,1.7,3961.4,3182.8,5452.7,14567.4,2025,362.843137
6,38.9,1935.2,1352.5,402.2,3049.8,0.9,1.7,3972.5,3198.3,5424.6,14830.0,2026,385.599940
7,39.4,1925.2,1381.0,406.0,3109.8,0.9,1.6,3983.6,3213.9,5396.5,15092.7,2027,472.677291
8,39.8,1915.1,1409.6,409.7,3169.8,0.9,1.6,3994.7,3229.4,5368.4,15355.4,2028,516.128459
9,40.2,1905.0,1438.1,413.5,3229.7,0.8,1.6,4005.8,3244.9,5340.3,15618.1,2029,566.352309


In [226]:
forecast_bed_2_flat_covid['St Kilda, St Kilda South, St Kilda West']

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop,Alleged_Offender_Incident_Rate,Victimisation_Rate,Offence_Count,year,Median_rental_price
0,36.3,1995.7,1181.3,379.5,2690.0,1.0,1.7,3905.9,3105.2,5593.1,13253.9,2020,409.881249
1,36.8,1985.7,1209.8,383.3,2750.0,1.0,1.7,3917.0,3120.7,5565.0,13516.6,2021,420.707281
2,37.2,1975.6,1238.4,387.0,2810.0,1.0,1.7,3928.1,3136.2,5536.9,13779.3,2022,436.580748
3,37.6,1965.5,1266.9,390.8,2869.9,1.0,1.7,3939.2,3151.8,5508.8,14042.0,2023,458.930775
4,38.1,1955.4,1295.4,394.6,2929.9,0.9,1.7,3950.3,3167.3,5480.7,14304.7,2024,490.391740
5,38.5,1945.3,1324.0,398.4,2989.9,0.9,1.7,3961.4,3182.8,5452.7,14567.4,2025,529.498973
6,38.9,1935.2,1352.5,402.2,3049.8,0.9,1.7,3972.5,3198.3,5424.6,14830.0,2026,578.455647
7,39.4,1925.2,1381.0,406.0,3109.8,0.9,1.6,3983.6,3213.9,5396.5,15092.7,2027,738.763641
8,39.8,1915.1,1409.6,409.7,3169.8,0.9,1.6,3994.7,3229.4,5368.4,15355.4,2028,828.797801
9,40.2,1905.0,1438.1,413.5,3229.7,0.8,1.6,4005.8,3244.9,5340.3,15618.1,2029,933.476136


### Getting growth rates by suburbs

#### Prediction without Covid

In [66]:
df_1bed_flat_no_covid = impute_missing(df_1bed_flat_no_covid, "-1", pred_cols)
bed_2_flat_no_covid = impute_missing(bed_2_flat_no_covid, "-1", pred_cols)
bed_2_house_no_covid = impute_missing(bed_2_house_no_covid, "-1", pred_cols)
bed_3_flat_no_covid = impute_missing(bed_3_flat_no_covid, "-1", pred_cols)
bed_3_house_no_covid = impute_missing(bed_3_house_no_covid, "-1", pred_cols)
bed_4_house_no_covid = impute_missing(bed_4_house_no_covid, "-1", pred_cols)

In [67]:
df_1bed_flat_no_covid_w_pop_cleaned = df_1bed_flat_no_covid.dropna()
bed_2_flat_no_covid_w_pop_cleaned = bed_2_flat_no_covid.dropna()
bed_2_house_no_covid_w_pop_cleaned = bed_2_house_no_covid.dropna()
bed_3_flat_no_covid_w_pop_cleaned = bed_3_flat_no_covid.dropna()
bed_3_house_no_covid_w_pop_cleaned = bed_3_house_no_covid.dropna()
bed_4_house_no_covid_w_pop_cleaned = bed_4_house_no_covid.dropna()

In [68]:
df_1bed_flat_no_covid_w_pop_cleaned

,Count_Mar2013,Median_Mar2013,Count_Jun2013,Median_Jun2013,Count_Sep2013,Median_Sep2013,Count_Dec2013,Median_Dec2013,Count_Mar2014,Median_Mar2014,...,Victimisation_Rate_2017,Victimisation_Rate_2018,Victimisation_Rate_2019,Offence_Count_2013,Offence_Count_2014,Offence_Count_2015,Offence_Count_2016,Offence_Count_2017,Offence_Count_2018,Offence_Count_2019
0,298,330.0,300,$330,320,$320,325,$320,328,320.0,...,6388.3,5620.4,5442.8,11054.0,11364.0,11915.0,12733.0,13427.0,12549.0,12778.0
1,180,305.0,178,$300,171,$300,173,$300,168,300.0,...,4981.9,4054.0,4181.7,8603.0,9444.0,9958.0,10204.0,11484.0,10191.0,10501.0
2,70,310.0,70,$310,60,$310,58,$310,56,313.0,...,8583.0,7697.4,7129.8,33752.0,890.0,33707.0,35452.0,37315.0,36659.0,35626.0
3,"2,280",320.0,"2,293",$320,"2,277",$325,"2,231",$325,"2,367",329.0,...,8583.0,7697.4,7129.8,33752.0,890.0,33707.0,35452.0,37315.0,36659.0,35626.0
4,"3,322",380.0,"3,522",$380,"3,700",$380,"3,904",$380,"4,045",375.0,...,6388.3,5620.4,5442.8,11054.0,11364.0,11915.0,12733.0,13427.0,12549.0,12778.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,40,198.0,41,$220,34,$220,31,$220,23,230.0,...,2988.4,2889.8,2694.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0
106,158,190.0,167,$200,199,$200,226,$200,248,210.0,...,4489.1,3743.5,4061.1,13223.0,890.0,14402.0,15288.0,16647.0,14308.0,14787.0
107,22,240.0,20,$240,17,$240,24,$205,21,194.0,...,2988.4,2889.8,2694.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0
108,24,218.0,27,$215,33,$215,31,$220,27,220.0,...,2988.4,2889.8,2694.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0


In [69]:
forecast_bed_1_flat_no_covid = get_forecast_all_suburbs(df_1bed_flat_no_covid_w_pop_cleaned, yrs[0:7], pred_cols)
forecast_bed_2_flat_no_covid = get_forecast_all_suburbs(bed_2_flat_no_covid_w_pop_cleaned, yrs[0:7], pred_cols)
forecast_bed_2_house_no_covid = get_forecast_all_suburbs(bed_2_house_no_covid_w_pop_cleaned, yrs[0:7], pred_cols)
forecast_bed_3_flat_no_covid = get_forecast_all_suburbs(bed_3_flat_no_covid_w_pop_cleaned, yrs[0:7], pred_cols)
forecast_bed_3_house_no_covid = get_forecast_all_suburbs(bed_3_house_no_covid_w_pop_cleaned, yrs[0:7], pred_cols)
forecast_bed_4_house_no_covid = get_forecast_all_suburbs(bed_4_house_no_covid_w_pop_cleaned, yrs[0:7], pred_cols)

In [70]:
forecast_bed_4_house_no_covid[bed_4_house.loc[0,'Suburb']]

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop,Alleged_Offender_Incident_Rate,Victimisation_Rate,Offence_Count,year,Median_rental_price
0,36.2,1986.2,1171.2,381.0,2662.7,1.0,1.7,3907.7,2761.2,5800.5,13553.4,2020,865.245461
1,36.6,1974.1,1197.6,385.1,2716.9,1.0,1.7,3919.2,2702.7,5816.0,13876.8,2021,875.408881
2,37.0,1962.0,1224.0,389.2,2771.1,1.0,1.7,3930.7,2644.2,5831.5,14200.1,2022,889.349104
3,37.4,1949.9,1250.4,393.3,2825.3,1.0,1.7,3942.2,2585.7,5847.1,14523.5,2023,907.668713
4,37.8,1937.8,1276.8,397.4,2879.5,0.9,1.7,3953.7,2527.2,5862.6,14846.9,2024,930.960305
5,38.2,1925.7,1303.2,401.5,2933.7,0.9,1.7,3965.2,2468.7,5878.1,15170.2,2025,959.824576
6,38.6,1913.6,1329.6,405.6,2987.9,0.9,1.7,3976.7,2410.2,5893.7,15493.6,2026,994.867970
7,39.0,1901.5,1356.0,409.7,3042.1,0.9,1.7,3988.2,2351.7,5909.2,15816.9,2027,1036.675748
8,39.4,1889.4,1382.4,413.8,3096.3,0.9,1.7,3999.7,2293.2,5924.7,16140.3,2028,1085.853591
9,39.8,1877.3,1408.8,417.9,3150.5,0.8,1.6,4011.2,2234.7,5940.3,16463.6,2029,1271.614113


# Analysis

In [21]:
CORR_COLS = ['Median_rental_price', 'Median_age_persons', 'Median_mortgage_repay_monthly', 'Median_tot_prsnl_inc_weekly', 'Median_tot_fam_inc_weekly','Average_num_psns_per_bedroom', 
             'Average_household_size', 'Pop']

In [22]:
# Function that get average correlation of given columns
def get_corr_avg_med(raw_df, yrs, pred_cols):
    plot_df = pd.DataFrame()

    for feature in CORR_COLS:
        df = pd.DataFrame()
        for i, row in raw_df.iterrows():
            rw = make_2D(row, yrs, pred_cols)
            df[str(i)] = rw[feature]
        avg_df = df.mean()
        plot_df[feature] = avg_df.values
        
    return plot_df

In [23]:
df_1bed_flat_corr = get_corr_avg_med(df_1bed_flat_w_pop_cleaned, yrs, pred_cols)
bed_2_flat_corr = get_corr_avg_med(bed_2_flat_w_pop_cleaned, yrs, pred_cols)
bed_2_house_corr = get_corr_avg_med(bed_2_house_w_pop_cleaned, yrs, pred_cols)
bed_3_flat_corr = get_corr_avg_med(bed_3_flat_w_pop_cleaned, yrs, pred_cols)
bed_3_house_corr = get_corr_avg_med(bed_3_house_w_pop_cleaned, yrs, pred_cols)
bed_4_house_corr = get_corr_avg_med(bed_4_house_w_pop_cleaned, yrs, pred_cols)

In [24]:
df_1bed_flat_corr

,Median_rental_price,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop
0,338.333333,35.029630,2025.992592,1095.696297,2510.125926,1.080000,1.758519,20223.888889
1,335.888889,37.200000,2427.022222,1159.429630,2947.066667,0.942963,2.058519,10199.666667
2,337.111111,25.459259,1888.911111,476.014815,1642.511111,1.080000,1.858519,18039.444444
3,343.111111,27.755556,2021.481481,604.703704,2292.096297,1.042963,2.180000,6492.333333
4,392.666667,35.029630,2025.992592,1095.696297,2510.125926,1.080000,1.758519,20223.888889
...,...,...,...,...,...,...,...,...
89,238.111111,50.200000,1776.614815,619.444444,1512.059259,0.764444,2.237037,6141.222222
90,225.555556,40.370370,1701.629630,670.333333,1727.918519,0.864444,2.400000,38189.888889
91,258.082707,40.555556,1623.651852,561.133333,1340.562963,0.864444,2.321481,9418.333333
92,258.195489,47.385185,1843.755556,657.666667,1708.429630,0.801481,2.300000,25205.444444


In [14]:
liveable_features = pd.read_csv('../data/curated/liveability.csv')
liveable_features

,Suburb,Postcode,NumParks,NumShoppingCentres,NumSchool,DstToStation
0,"St Kilda, St Kilda South, St Kilda West",3182,9,3,5,1444.0
1,"Armadale North, Armadale",3143,0,0,2,307.7
2,"Carlton South, Carlton",3053,3,0,2,1978.0
3,"Melbourne University, Parkville",3052,3,0,3,1139.4
4,"St Kilda, St Kilda South, St Kilda West",3182,9,3,5,1444.0
...,...,...,...,...,...,...
89,"Dromana, Arthurs Seat, Safety Beach",3936,1,1,3,21947.4
90,"Karingal Centre, Karingal, Frankston Heights, ...",3199,17,3,20,2250.4
91,"Tuerong, Hastings",3915,2,0,4,8203.2
92,Mornington,3931,15,3,7,13918.4


In [16]:
victim_rate  = bed_2_flat_w_pop_cleaned.iloc[:,150:159].copy()
victim_rate['Mean_Victimisation_Rate'] = victim_rate.mean(axis=1)

### Rate of victim per 100 people

In [17]:
victim_rate['Mean_Victimisation_Rate'] = victim_rate['Mean_Victimisation_Rate'] / 1000

In [18]:
victim_rate['Mean_Victimisation_Rate']

0      5.677344
1      4.411300
2      8.026456
3      8.026456
4      5.677344
         ...   
105    2.889044
106    4.005789
107    2.889044
108    2.889044
109    2.764978
Name: Mean_Victimisation_Rate, Length: 94, dtype: float64

In [83]:
victim_rate['Mean_Victimisation_Rate'].to_csv('../data/test.csv')

In [19]:
victim_rate

,Victimisation_Rate_2013,Victimisation_Rate_2014,Victimisation_Rate_2015,Victimisation_Rate_2016,Victimisation_Rate_2017,Victimisation_Rate_2018,Victimisation_Rate_2019,Victimisation_Rate_2020,Victimisation_Rate_2021,Mean_Victimisation_Rate
0,5785.3,5318.3,5530.1,6083.2,6388.3,5620.4,5442.8,5662.4,5265.3,5.677344
1,4168.6,4049.3,4180.4,4602.6,4981.9,4054.0,4181.7,5084.9,4398.3,4.411300
2,9957.8,8590.0,7733.5,8526.6,8583.0,7697.4,7129.8,7790.8,6229.2,8.026456
3,9957.8,8590.0,7733.5,8526.6,8583.0,7697.4,7129.8,7790.8,6229.2,8.026456
4,5785.3,5318.3,5530.1,6083.2,6388.3,5620.4,5442.8,5662.4,5265.3,5.677344
...,...,...,...,...,...,...,...,...,...,...
105,3107.1,3134.8,2760.2,3122.1,2988.4,2889.8,2694.2,3061.6,2243.2,2.889044
106,4282.8,4063.7,3680.2,4250.1,4489.1,3743.5,4061.1,4162.5,3319.1,4.005789
107,3107.1,3134.8,2760.2,3122.1,2988.4,2889.8,2694.2,3061.6,2243.2,2.889044
108,3107.1,3134.8,2760.2,3122.1,2988.4,2889.8,2694.2,3061.6,2243.2,2.889044


In [76]:
liveable_features.shape

(94, 6)

In [77]:
liveable_features = liveable_features.join(victim_rate['Mean_Victimisation_Rate'])
liveable_features.isna().sum()

Suburb                      0
Postcode                    0
NumParks                    0
NumShoppingCentres          0
NumSchool                   0
DstToStation                0
Mean_Victimisation_Rate    14
dtype: int64

In [71]:
liveable_features

,Suburb,Postcode,NumParks,NumShoppingCentres,NumSchool,DstToStation
0,"St Kilda, St Kilda South, St Kilda West",3182,9,3,5,1444.0
1,"Armadale North, Armadale",3143,0,0,2,307.7
2,"Carlton South, Carlton",3053,3,0,2,1978.0
3,"Melbourne University, Parkville",3052,3,0,3,1139.4
4,"St Kilda, St Kilda South, St Kilda West",3182,9,3,5,1444.0
...,...,...,...,...,...,...
89,"Dromana, Arthurs Seat, Safety Beach",3936,1,1,3,21947.4
90,"Karingal Centre, Karingal, Frankston Heights, ...",3199,17,3,20,2250.4
91,"Tuerong, Hastings",3915,2,0,4,8203.2
92,Mornington,3931,15,3,7,13918.4


Column was migrated in Excel

In [13]:
final_liveable = pd.read_csv('../data/curated/final_liveability.csv')
final_liveable

,Suburb,Postcode,NumParks,NumShoppingCentres,NumSchool,DstToStation,Mean_Victimisation_Rate
0,"St Kilda, St Kilda South, St Kilda West",3182,9,3,5,1444.0,5.677344
1,"Armadale North, Armadale",3143,0,0,2,307.7,4.411300
2,"Carlton South, Carlton",3053,3,0,2,1978.0,8.026456
3,"Melbourne University, Parkville",3052,3,0,3,1139.4,8.026456
4,"St Kilda, St Kilda South, St Kilda West",3182,9,3,5,1444.0,5.677344
...,...,...,...,...,...,...,...
89,"Dromana, Arthurs Seat, Safety Beach",3936,1,1,3,21947.4,2.889044
90,"Karingal Centre, Karingal, Frankston Heights, ...",3199,17,3,20,2250.4,4.005789
91,"Tuerong, Hastings",3915,2,0,4,8203.2,2.889044
92,Mornington,3931,15,3,7,13918.4,2.889044


In [128]:
liveable_features_sorted_on_all = final_liveable.sort_values(['Mean_Victimisation_Rate', 'NumSchool', 'DstToStation', 
                                                                 'NumShoppingCentres', 'NumParks'],
              ascending = [True, False, True, False, False])

In [129]:
liveable_features_sorted_on_all.head(10)

,Suburb,Postcode,NumParks,NumShoppingCentres,NumSchool,DstToStation,Mean_Victimisation_Rate
20,Bulleen,3105,4,1,4,4610.8,1.788400
26,Doncaster,3108,10,2,4,4768.2,1.788400
82,"Hoddles Creek, Wandin North, Beenak, Launching...",3139,7,0,7,13490.6,2.019356
18,"Laburnum, Blackburn South, Blackburn, Blackbur...",3130,18,1,11,1226.0,2.344311
19,"Box Hill, Box Hill Central, Wattle Park, Houst...",3128,7,1,4,2168.7,2.344311
31,"Forest Hill, Nunawading, Brentford Square",3131,13,2,4,2846.1,2.344311
29,"Cotham, Kew",3101,8,0,11,2120.6,2.345856
27,"Glenferrie South, Hawthorn North, Hawthorn Wes...",3122,4,0,8,234.6,2.345856
17,"Deepdene, Balwyn East, Balwyn, Stradbroke Park",3103,11,0,7,1759.1,2.345856
22,"Camberwell West, Camberwell South, Hartwell, C...",3124,7,1,5,709.8,2.345856


In [130]:
liveable_features_sorted_on_all.to_csv('../data/curated/final_liveability_sorted.csv', index = False)

In [14]:
final_liveable.iloc[:,3:7]

,NumShoppingCentres,NumSchool,DstToStation,Mean_Victimisation_Rate
0,3,5,1444.0,5.677344
1,0,2,307.7,4.411300
2,0,2,1978.0,8.026456
3,0,3,1139.4,8.026456
4,3,5,1444.0,5.677344
...,...,...,...,...
89,1,3,21947.4,2.889044
90,3,20,2250.4,4.005789
91,0,4,8203.2,2.889044
92,3,7,13918.4,2.889044


In [25]:
df_1bed_flat_corr = df_1bed_flat_corr.join(final_liveable.iloc[:,3:7])
bed_2_flat_corr = bed_2_flat_corr.join(final_liveable.iloc[:,3:7])
bed_2_house_corr = bed_2_house_corr.join(final_liveable.iloc[:,3:7])
bed_3_flat_corr = bed_3_flat_corr.join(final_liveable.iloc[:,3:7])
bed_3_house_corr = bed_3_house_corr.join(final_liveable.iloc[:,3:7])
bed_4_house_corr = bed_4_house_corr.join(final_liveable.iloc[:,3:7])

In [26]:
df_1bed_flat_corr.corr()

,Median_rental_price,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop,NumShoppingCentres,NumSchool,DstToStation,Mean_Victimisation_Rate
Median_rental_price,1.000000,-0.175918,0.440109,0.466992,0.425493,0.380956,-0.493569,-0.006802,0.147395,-0.246494,-0.245253,0.323330
Median_age_persons,-0.175918,1.000000,0.374819,0.091389,0.244714,-0.749732,0.182155,-0.059187,-0.259163,-0.077630,0.163728,-0.541733
Median_mortgage_repay_monthly,0.440109,0.374819,1.000000,0.679648,0.890023,-0.086221,-0.244461,-0.138137,-0.188930,-0.372021,-0.158845,-0.049646
Median_tot_prsnl_inc_weekly,0.466992,0.091389,0.679648,1.000000,0.896528,0.211053,-0.585525,-0.166718,-0.113988,-0.352354,-0.124141,0.287886
Median_tot_fam_inc_weekly,0.425493,0.244714,0.890023,0.896528,1.000000,0.044297,-0.410166,-0.166974,-0.174876,-0.376677,-0.127597,0.119903
Average_num_psns_per_bedroom,0.380956,-0.749732,-0.086221,0.211053,0.044297,1.000000,-0.487071,-0.047366,0.301978,-0.134123,-0.242786,0.741805
Average_household_size,-0.493569,0.182155,-0.244461,-0.585525,-0.410166,-0.487071,1.000000,0.109385,-0.149048,0.573909,0.142880,-0.578598
Pop,-0.006802,-0.059187,-0.138137,-0.166718,-0.166974,-0.047366,0.109385,1.000000,0.300773,0.388226,0.035357,-0.052378
NumShoppingCentres,0.147395,-0.259163,-0.188930,-0.113988,-0.174876,0.301978,-0.149048,0.300773,1.000000,0.093108,-0.021137,0.233904
NumSchool,-0.246494,-0.077630,-0.372021,-0.352354,-0.376677,-0.134123,0.573909,0.388226,0.093108,1.000000,0.086059,-0.206268


In [27]:
bed_2_flat_corr.corr()

,Median_rental_price,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop,NumShoppingCentres,NumSchool,DstToStation,Mean_Victimisation_Rate
Median_rental_price,1.000000,-0.259532,0.480502,0.568662,0.517442,0.512306,-0.652795,-0.009964,0.168028,-0.385283,-0.193693,0.442995
Median_age_persons,-0.259532,1.000000,0.374819,0.091389,0.244714,-0.749732,0.182155,-0.059187,-0.259163,-0.077630,0.163728,-0.541733
Median_mortgage_repay_monthly,0.480502,0.374819,1.000000,0.679648,0.890023,-0.086221,-0.244461,-0.138137,-0.188930,-0.372021,-0.158845,-0.049646
Median_tot_prsnl_inc_weekly,0.568662,0.091389,0.679648,1.000000,0.896528,0.211053,-0.585525,-0.166718,-0.113988,-0.352354,-0.124141,0.287886
Median_tot_fam_inc_weekly,0.517442,0.244714,0.890023,0.896528,1.000000,0.044297,-0.410166,-0.166974,-0.174876,-0.376677,-0.127597,0.119903
Average_num_psns_per_bedroom,0.512306,-0.749732,-0.086221,0.211053,0.044297,1.000000,-0.487071,-0.047366,0.301978,-0.134123,-0.242786,0.741805
Average_household_size,-0.652795,0.182155,-0.244461,-0.585525,-0.410166,-0.487071,1.000000,0.109385,-0.149048,0.573909,0.142880,-0.578598
Pop,-0.009964,-0.059187,-0.138137,-0.166718,-0.166974,-0.047366,0.109385,1.000000,0.300773,0.388226,0.035357,-0.052378
NumShoppingCentres,0.168028,-0.259163,-0.188930,-0.113988,-0.174876,0.301978,-0.149048,0.300773,1.000000,0.093108,-0.021137,0.233904
NumSchool,-0.385283,-0.077630,-0.372021,-0.352354,-0.376677,-0.134123,0.573909,0.388226,0.093108,1.000000,0.086059,-0.206268


In [28]:
bed_2_house_corr.corr()

,Median_rental_price,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,Pop,NumShoppingCentres,NumSchool,DstToStation,Mean_Victimisation_Rate
Median_rental_price,1.000000,-0.070389,0.459774,0.492796,0.525199,0.259819,-0.489672,-0.070680,0.092480,-0.223907,-0.169064,0.177406
Median_age_persons,-0.070389,1.000000,0.374819,0.091389,0.244714,-0.749732,0.182155,-0.059187,-0.259163,-0.077630,0.163728,-0.541733
Median_mortgage_repay_monthly,0.459774,0.374819,1.000000,0.679648,0.890023,-0.086221,-0.244461,-0.138137,-0.188930,-0.372021,-0.158845,-0.049646
Median_tot_prsnl_inc_weekly,0.492796,0.091389,0.679648,1.000000,0.896528,0.211053,-0.585525,-0.166718,-0.113988,-0.352354,-0.124141,0.287886
Median_tot_fam_inc_weekly,0.525199,0.244714,0.890023,0.896528,1.000000,0.044297,-0.410166,-0.166974,-0.174876,-0.376677,-0.127597,0.119903
Average_num_psns_per_bedroom,0.259819,-0.749732,-0.086221,0.211053,0.044297,1.000000,-0.487071,-0.047366,0.301978,-0.134123,-0.242786,0.741805
Average_household_size,-0.489672,0.182155,-0.244461,-0.585525,-0.410166,-0.487071,1.000000,0.109385,-0.149048,0.573909,0.142880,-0.578598
Pop,-0.070680,-0.059187,-0.138137,-0.166718,-0.166974,-0.047366,0.109385,1.000000,0.300773,0.388226,0.035357,-0.052378
NumShoppingCentres,0.092480,-0.259163,-0.188930,-0.113988,-0.174876,0.301978,-0.149048,0.300773,1.000000,0.093108,-0.021137,0.233904
NumSchool,-0.223907,-0.077630,-0.372021,-0.352354,-0.376677,-0.134123,0.573909,0.388226,0.093108,1.000000,0.086059,-0.206268


In [62]:
bed_4_house_corr.corr()

,Median_rental_price,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,NumShoppingCentres,NumSchool,DstToStation,Mean_Victimisation_Rate
Median_rental_price,1.000000,0.157788,0.645956,0.595986,0.694627,0.017586,-0.344128,-0.301739,-0.273974,-0.136177,0.035388
Median_age_persons,0.157788,1.000000,0.374819,0.091389,0.244714,-0.749732,0.182155,-0.259163,-0.077630,0.163728,-0.541733
Median_mortgage_repay_monthly,0.645956,0.374819,1.000000,0.679648,0.890023,-0.086221,-0.244461,-0.188930,-0.372021,-0.158845,-0.049646
Median_tot_prsnl_inc_weekly,0.595986,0.091389,0.679648,1.000000,0.896528,0.211053,-0.585525,-0.113988,-0.352354,-0.124141,0.287886
Median_tot_fam_inc_weekly,0.694627,0.244714,0.890023,0.896528,1.000000,0.044297,-0.410166,-0.174876,-0.376677,-0.127597,0.119903
Average_num_psns_per_bedroom,0.017586,-0.749732,-0.086221,0.211053,0.044297,1.000000,-0.487071,0.301978,-0.134123,-0.242786,0.741805
Average_household_size,-0.344128,0.182155,-0.244461,-0.585525,-0.410166,-0.487071,1.000000,-0.149048,0.573909,0.142880,-0.578598
NumShoppingCentres,-0.301739,-0.259163,-0.188930,-0.113988,-0.174876,0.301978,-0.149048,1.000000,0.093108,-0.021137,0.233904
NumSchool,-0.273974,-0.077630,-0.372021,-0.352354,-0.376677,-0.134123,0.573909,0.093108,1.000000,0.086059,-0.206268
DstToStation,-0.136177,0.163728,-0.158845,-0.124141,-0.127597,-0.242786,0.142880,-0.021137,0.086059,1.000000,-0.137639


In [23]:
df_1bed_flat_corr.corr().to_csv("../data/curated/corr/bed_1_flat_corr.csv")
bed_2_flat_corr.corr().to_csv("../data/curated/corr/bed_2_flat_corr.csv")
bed_2_house_corr.corr().to_csv("../data/curated/corr/bed_2_house_corr.csv")
bed_3_flat_corr.corr().to_csv("../data/curated/corr/bed_3_flat_corr.csv")
bed_3_house_corr.corr().to_csv("../data/curated/corr/bed_3_house_corr.csv")
bed_4_house_corr.corr().to_csv("../data/curated/corr/bed_4_house_corr.csv")

# Aggregate by Victoria

In [94]:
agg_yrs = [2013,2014,2015,2016,2017,2018,2019,2020,2021]

In [95]:
df_1bed_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_1_bed_flat_monetary_2013-2021_cleaned.csv')
bed_2_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_flat_monetary_2013-2021_cleaned.csv')
bed_2_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_house_monetary_2013-2021_cleaned.csv')
bed_3_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_flat_monetary_2013-2021_cleaned.csv')
bed_3_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_house_monetary_2013-2021_cleaned.csv')
bed_4_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_4_house_monetary_2013-2021_cleaned.csv')

In [90]:
df_1bed_flat

,Count_Mar2013,Median_Mar2013,Count_Jun2013,Median_Jun2013,Count_Sep2013,Median_Sep2013,Count_Dec2013,Median_Dec2013,Count_Mar2014,Median_Mar2014,...,Victimisation_Rate_2021,Offence_Count_2013,Offence_Count_2014,Offence_Count_2015,Offence_Count_2016,Offence_Count_2017,Offence_Count_2018,Offence_Count_2019,Offence_Count_2020,Offence_Count_2021
0,298,330.0,300,$330,320,$320,325,$320,328,320.0,...,5265.3,11054.0,11364.0,11915.0,12733.0,13427.0,12549.0,12778.0,12965.0,13408.0
1,180,305.0,178,$300,171,$300,173,$300,168,300.0,...,4398.3,8603.0,9444.0,9958.0,10204.0,11484.0,10191.0,10501.0,12090.0,11635.0
2,70,310.0,70,$310,60,$310,58,$310,56,313.0,...,6229.2,33752.0,890.0,33707.0,35452.0,37315.0,36659.0,35626.0,36359.0,35712.0
3,"2,280",320.0,"2,293",$320,"2,277",$325,"2,231",$325,"2,367",329.0,...,6229.2,33752.0,890.0,33707.0,35452.0,37315.0,36659.0,35626.0,36359.0,35712.0
4,"3,322",380.0,"3,522",$380,"3,700",$380,"3,904",$380,"4,045",375.0,...,5265.3,11054.0,11364.0,11915.0,12733.0,13427.0,12549.0,12778.0,12965.0,13408.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,40,198.0,41,$220,34,$220,31,$220,23,230.0,...,2243.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0,11317.0,10774.0
90,158,190.0,167,$200,199,$200,226,$200,248,210.0,...,3319.1,13223.0,890.0,14402.0,15288.0,16647.0,14308.0,14787.0,14942.0,14157.0
91,22,240.0,20,$240,17,$240,24,$205,21,194.0,...,2243.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0,11317.0,10774.0
92,24,218.0,27,$215,33,$215,31,$220,27,220.0,...,2243.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0,11317.0,10774.0


In [98]:
df_1bed_flat_new = impute_missing(df_1bed_flat, np.nan, pred_cols[:9])
bed_2_flat_new = impute_missing(bed_2_flat, np.nan, pred_cols[:9])
bed_2_house_new = impute_missing(bed_2_house, np.nan, pred_cols[:9])
bed_3_flat_new = impute_missing(bed_3_flat, np.nan, pred_cols[:9])
bed_3_house_new = impute_missing(bed_3_house, np.nan, pred_cols[:9])
bed_4_house_new = impute_missing(bed_4_house, np.nan, pred_cols[:9])

In [99]:
df_1bed_flat_new = df_1bed_flat_new.dropna()
bed_2_flat_new = bed_2_flat_new.dropna()
bed_2_house_new = bed_2_house_new.dropna()
bed_3_flat_new = bed_3_flat_new.dropna()
bed_3_house_new = bed_3_house_new.dropna()
bed_4_house_new = bed_4_house_new.dropna()

In [101]:
df_1bed_flat_new

,Count_Mar2013,Median_Mar2013,Count_Jun2013,Median_Jun2013,Count_Sep2013,Median_Sep2013,Count_Dec2013,Median_Dec2013,Count_Mar2014,Median_Mar2014,...,Victimisation_Rate_2021,Offence_Count_2013,Offence_Count_2014,Offence_Count_2015,Offence_Count_2016,Offence_Count_2017,Offence_Count_2018,Offence_Count_2019,Offence_Count_2020,Offence_Count_2021
0,298,330.0,300,$330,320,$320,325,$320,328,320.0,...,5265.3,11054.0,11364.0,11915.0,12733.0,13427.0,12549.0,12778.0,12965.0,13408.0
1,180,305.0,178,$300,171,$300,173,$300,168,300.0,...,4398.3,8603.0,9444.0,9958.0,10204.0,11484.0,10191.0,10501.0,12090.0,11635.0
2,70,310.0,70,$310,60,$310,58,$310,56,313.0,...,6229.2,33752.0,890.0,33707.0,35452.0,37315.0,36659.0,35626.0,36359.0,35712.0
3,"2,280",320.0,"2,293",$320,"2,277",$325,"2,231",$325,"2,367",329.0,...,6229.2,33752.0,890.0,33707.0,35452.0,37315.0,36659.0,35626.0,36359.0,35712.0
4,"3,322",380.0,"3,522",$380,"3,700",$380,"3,904",$380,"4,045",375.0,...,5265.3,11054.0,11364.0,11915.0,12733.0,13427.0,12549.0,12778.0,12965.0,13408.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,40,198.0,41,$220,34,$220,31,$220,23,230.0,...,2243.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0,11317.0,10774.0
90,158,190.0,167,$200,199,$200,226,$200,248,210.0,...,3319.1,13223.0,890.0,14402.0,15288.0,16647.0,14308.0,14787.0,14942.0,14157.0
91,22,240.0,20,$240,17,$240,24,$205,21,194.0,...,2243.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0,11317.0,10774.0
92,24,218.0,27,$215,33,$215,31,$220,27,220.0,...,2243.2,9154.0,9922.0,10231.0,11427.0,11745.0,11023.0,10875.0,11317.0,10774.0


In [140]:
# Function returns average of a feautre by year historical
def get_avg_by_year(df, yrs, pred_cols, target_column):
    avg = {}
    for i,rw in df.iterrows():
        X_df = make_2D(rw, yrs, pred_cols)
        for j,row in X_df.iterrows():
            if row['year'] in avg.keys():
                avg[row['year']] += row[target_column]
            else:
                avg[row['year']] = row[target_column]
           
    avg = {x: avg[x]/df.shape[0] for x in avg}
    return avg

In [92]:
pred_cols

['Median_Mar',
 'Median_age_persons',
 'Median_mortgage_repay_monthly',
 'Median_tot_prsnl_inc_weekly',
 'Median_rent_weekly',
 'Median_tot_fam_inc_weekly',
 'Average_num_psns_per_bedroom',
 'Average_household_size',
 'Pop',
 'Alleged_Offender_Incident_Rate',
 'Victimisation_Rate',
 'Offence_Count']

In [141]:
df_1bed_flat_avg_by_yr = get_avg_by_year(df_1bed_flat_new, agg_yrs, pred_cols, 'Median_rental_price')
bed_2_flat_avg_by_yr = get_avg_by_year(bed_2_flat_new, agg_yrs, pred_cols, 'Median_rental_price')
bed_2_house_avg_by_yr = get_avg_by_year(bed_2_house_new, agg_yrs, pred_cols, 'Median_rental_price')
bed_3_flat_avg_by_yr = get_avg_by_year(bed_3_flat_new, agg_yrs, pred_cols, 'Median_rental_price')
bed_3_house_avg_by_yr = get_avg_by_year(bed_3_house_new, agg_yrs, pred_cols, 'Median_rental_price')
bed_4_house_avg_by_yr = get_avg_by_year(bed_4_house_new, agg_yrs, pred_cols, 'Median_rental_price')

In [109]:
df_1bed_flat_avg_by_yr

{2013.0: 264.6839036788596,
 2014.0: 271.44020542920026,
 2015.0: 277.71276595744683,
 2016.0: 287.60479502306555,
 2017.0: 296.6284122842232,
 2018.0: 311.51918137871536,
 2019.0: 320.1636487098234,
 2020.0: 325.5058609490406,
 2021.0: 313.2780097391706}

In [162]:
bed_4_house_avg_by_yr

{2011.0: 519.2446808510638,
 2012.0: 551.9148936170212,
 2013.0: 552.2765957446809,
 2014.0: 574.7978723404256,
 2015.0: 596.8191489361702,
 2016.0: 610.1489361702128,
 2017.0: 632.7553191489362,
 2018.0: 651.2978723404256,
 2019.0: 666.6063829787234,
 2020.0: 676.5,
 2021.0: 661.1170212765958}

In [124]:
# Function returns average of a feautre by year future
def get_avg_by_year_pred(forecast_dict, df, target_column):
    pred_dict ={}
    for sub in forecast_dict.keys():
        if sub in df.loc[:,'Suburb'].tolist():
    #         print(forecast_bed_4_house_covid[sub])
            curr_df = forecast_dict[sub]
            for j,row in curr_df.iterrows():
                if row['year'] in pred_dict.keys():
                    pred_dict[row['year']] += row[target_column]
                else:
                    pred_dict[row['year']] = row[target_column]

    pred_dict = {x: pred_dict[x]/df.shape[0] for x in pred_dict}
    return pred_dict

In [133]:
df_1bed_flat_avg_by_yr_pred = get_avg_by_year_pred(forecast_bed_1_flat_covid , df_1bed_flat_new, 'Median_rental_price')
bed_2_flat_avg_by_yr_pred = get_avg_by_year_pred(forecast_bed_2_flat_covid , bed_2_flat_new , 'Median_rental_price')
bed_2_house_avg_by_yr_pred = get_avg_by_year_pred(forecast_bed_2_house_covid , bed_2_house_new , 'Median_rental_price')
bed_3_flat_avg_by_yr_pred = get_avg_by_year_pred(forecast_bed_3_flat_covid , bed_3_flat_new , 'Median_rental_price')
bed_3_house_avg_by_yr_pred = get_avg_by_year_pred(forecast_bed_3_house_covid , bed_3_house_new , 'Median_rental_price')
bed_4_house_avg_by_yr_pred = get_avg_by_year_pred(forecast_bed_4_house_covid , bed_4_house_new , 'Median_rental_price')

df_1bed_flat_avg_by_yr_pred_no_covid = get_avg_by_year_pred(forecast_bed_1_flat_no_covid , df_1bed_flat_new , 'Median_rental_price')
bed_2_flat_avg_by_yr_pred_no_covid = get_avg_by_year_pred(forecast_bed_2_flat_no_covid , bed_2_flat_new , 'Median_rental_price')
bed_2_house_avg_by_yr_pred_no_covid = get_avg_by_year_pred(forecast_bed_2_house_no_covid , bed_2_house_new , 'Median_rental_price')
bed_3_flat_avg_by_yr_pred_no_covid = get_avg_by_year_pred(forecast_bed_3_flat_no_covid , bed_3_flat_new , 'Median_rental_price')
bed_3_house_avg_by_yr_pred_no_covid = get_avg_by_year_pred(forecast_bed_3_house_no_covid , bed_3_house_new , 'Median_rental_price')
bed_4_house_avg_by_yr_pred_no_covid = get_avg_by_year_pred(forecast_bed_4_house_no_covid , bed_4_house_new , 'Median_rental_price')

In [134]:
df_1bed_flat_avg_by_yr_pred

{2020.0: 287.2319433584929,
 2021.0: 296.2454970796875,
 2022.0: 310.135547587617,
 2023.0: 330.8943197663635,
 2024.0: 362.2547087781612,
 2025.0: 401.95760804611865,
 2026.0: 458.15852633823835,
 2027.0: 546.69334948357,
 2028.0: 623.5865933865974,
 2029.0: 733.5717319104857,
 2030.0: 843.9108501227768}

In [127]:
bed_4_house_avg_by_yr_pred_no_covid

{2020.0: 607.7660637909087,
 2021.0: 647.2201514076232,
 2022.0: 701.6621422370988,
 2023.0: 774.6060553893565,
 2024.0: 897.0291398261554,
 2025.0: 1022.6328375490111,
 2026.0: 1186.2859019077175,
 2027.0: 1378.401918739042,
 2028.0: 1600.2143667380853,
 2029.0: 2045.9099763608112,
 2030.0: 2369.9672010849304}

In [128]:
final_agg_vic = {}
for key in bed_4_house_avg_by_yr.keys():
    final_agg_vic[key] = (df_1bed_flat_avg_by_yr[key] + bed_2_flat_avg_by_yr[key] + bed_2_house_avg_by_yr[key])/3
    
final_agg_vic

{2013.0: 323.3662657652937,
 2014.0: 330.5396522763254,
 2015.0: 338.05718085106383,
 2016.0: 346.8951430851154,
 2017.0: 358.04635019403185,
 2018.0: 373.9248335092172,
 2019.0: 384.56873396710427,
 2020.0: 390.54450684116955,
 2021.0: 376.3657195584469}

In [129]:
final_agg_vic_pred = {}
for key in bed_2_flat_avg_by_yr_pred.keys():
    final_agg_vic_pred[key] = (df_1bed_flat_avg_by_yr_pred_no_covid[key] + bed_2_flat_avg_by_yr_pred_no_covid[key] + bed_2_house_avg_by_yr_pred_no_covid[key])/3
    
final_agg_vic_pred

{2020.0: 373.8204298623757,
 2021.0: 412.01884655796744,
 2022.0: 464.2694150593563,
 2023.0: 534.5520846424512,
 2024.0: 647.4794521944992,
 2025.0: 766.6290587770164,
 2026.0: 922.7709591806856,
 2027.0: 1105.109792435291,
 2028.0: 1315.5429261490833,
 2029.0: 1759.1677613760905,
 2030.0: 2069.212208824116}

In [142]:
df_1bed_flat_avg_by_yr_income = get_avg_by_year(df_1bed_flat_new, agg_yrs, pred_cols, 'Median_tot_prsnl_inc_weekly')
bed_2_flat_avg_by_yr_income = get_avg_by_year(bed_2_flat_new, agg_yrs, pred_cols, 'Median_tot_prsnl_inc_weekly')
bed_2_house_avg_by_yr_income = get_avg_by_year(bed_2_house_new, agg_yrs, pred_cols, 'Median_tot_prsnl_inc_weekly')
bed_3_flat_avg_by_yr_income = get_avg_by_year(bed_3_flat_new, agg_yrs, pred_cols, 'Median_tot_prsnl_inc_weekly')
bed_3_house_avg_by_yr_income = get_avg_by_year(bed_3_house_new, agg_yrs, pred_cols, 'Median_tot_prsnl_inc_weekly')
bed_4_house_avg_by_yr_income = get_avg_by_year(bed_4_house_new, agg_yrs, pred_cols, 'Median_tot_prsnl_inc_weekly')



df_1bed_flat_avg_by_yr_pred_income = get_avg_by_year_pred(forecast_bed_1_flat_covid , df_1bed_flat_new, 'Median_tot_prsnl_inc_weekly')
bed_2_flat_avg_by_yr_pred_income = get_avg_by_year_pred(forecast_bed_2_flat_covid , bed_2_flat_new , 'Median_tot_prsnl_inc_weekly')
bed_2_house_avg_by_yr_pred_income = get_avg_by_year_pred(forecast_bed_2_house_covid , bed_2_house_new , 'Median_tot_prsnl_inc_weekly')
bed_3_flat_avg_by_yr_pred_income = get_avg_by_year_pred(forecast_bed_3_flat_covid , bed_3_flat_new , 'Median_tot_prsnl_inc_weekly')
bed_3_house_avg_by_yr_pred_income = get_avg_by_year_pred(forecast_bed_3_house_covid , bed_3_house_new , 'Median_tot_prsnl_inc_weekly')
bed_4_house_avg_by_yr_pred_income = get_avg_by_year_pred(forecast_bed_4_house_covid , bed_4_house_new , 'Median_tot_prsnl_inc_weekly')

In [136]:
df_1bed_flat_avg_by_yr_pred_income

{2020.0: 835.245744680851,
 2021.0: 863.2021276595751,
 2022.0: 891.1585106382977,
 2023.0: 919.1244680851065,
 2024.0: 947.0851063829786,
 2025.0: 975.0425531914893,
 2026.0: 1003.006382978723,
 2027.0: 1030.9638297872339,
 2028.0: 1058.9212765957445,
 2029.0: 1086.8861702127656,
 2030.0: 1114.8468085106383}

In [143]:
df_1bed_flat_avg_by_yr_income

{2013.0: 684.5429078138297,
 2014.0: 712.2673759085104,
 2015.0: 739.9918439904257,
 2016.0: 734.4574468085107,
 2017.0: 795.4407801734042,
 2018.0: 823.1652482553189,
 2019.0: 850.8897163340425,
 2020.0: 878.6141844287233,
 2021.0: 922.968085106383}

Median_age_persons
Median_mortgage_repay_monthly
Median_tot_prsnl_inc_weekly
Average_household_size
Population
Alleged_Offender_Incident_Rate
Victimization Rate
Offender Count

In [144]:
df_1bed_flat_avg_by_yr_age = get_avg_by_year(df_1bed_flat_new, agg_yrs, pred_cols, 'Median_age_persons')

df_1bed_flat_avg_by_yr_pred_age = get_avg_by_year_pred(forecast_bed_1_flat_covid , df_1bed_flat_new, 'Median_age_persons')

In [146]:
df_1bed_flat_avg_by_yr_pred_age

{2020.0: 36.22978723404258,
 2021.0: 36.41170212765959,
 2022.0: 36.576595744680844,
 2023.0: 36.73723404255319,
 2024.0: 36.923404255319134,
 2025.0: 37.082978723404274,
 2026.0: 37.26595744680853,
 2027.0: 37.447872340425505,
 2028.0: 37.612765957446804,
 2029.0: 37.77340425531915,
 2030.0: 37.95957446808514}

In [145]:
df_1bed_flat_avg_by_yr_age

{2013.0: 36.59574468106385,
 2014.0: 36.76595744702126,
 2015.0: 36.93617021297873,
 2016.0: 36.787234042553195,
 2017.0: 37.27659574489361,
 2018.0: 37.44680851085108,
 2019.0: 37.61702127680849,
 2020.0: 37.78723404276597,
 2021.0: 38.11702127659574}

In [148]:
df_1bed_flat_avg_by_yr_pop = get_avg_by_year(df_1bed_flat_new, agg_yrs, pred_cols, 'Pop')

df_1bed_flat_avg_by_yr_pred_pop = get_avg_by_year_pred(forecast_bed_1_flat_covid , df_1bed_flat_new, 'Pop')

In [149]:
df_1bed_flat_avg_by_yr_pop

{2013.0: 17527.73404255319,
 2014.0: 17759.457446808512,
 2015.0: 17991.212765957447,
 2016.0: 18222.978723404256,
 2017.0: 18454.691489361703,
 2018.0: 18686.425531914894,
 2019.0: 18918.18085106383,
 2020.0: 19149.882978723403,
 2021.0: 19531.91489361702}

In [150]:
df_1bed_flat_avg_by_yr_pred_pop

{2020.0: 18293.528723404255,
 2021.0: 18525.1755319149,
 2022.0: 18756.83404255319,
 2023.0: 18988.479787234046,
 2024.0: 19220.12872340426,
 2025.0: 19451.78829787234,
 2026.0: 19683.431914893612,
 2027.0: 19915.079787234044,
 2028.0: 20146.739361702126,
 2029.0: 20378.3840425532,
 2030.0: 20610.031914893614}

In [158]:
df_1bed_flat_avg_by_yr = get_avg_by_year(df_1bed_flat_new, agg_yrs, pred_cols, 'Median_mortgage_repay_monthly')

df_1bed_flat_avg_by_yr_pred = get_avg_by_year_pred(forecast_bed_1_flat_covid , df_1bed_flat_new, 'Median_mortgage_repay_monthly')

In [159]:
df_1bed_flat_avg_by_yr

{2013.0: 1985.9421985744684,
 2014.0: 2005.875177297873,
 2015.0: 2025.8081560212765,
 2016.0: 1971.0425531914893,
 2017.0: 2065.674113468086,
 2018.0: 2085.6070921914893,
 2019.0: 2105.5400709148944,
 2020.0: 2125.4730496382977,
 2021.0: 2182.755319148936}

In [160]:
df_1bed_flat_avg_by_yr_pred

{2020.0: 2036.6297872340435,
 2021.0: 2059.58829787234,
 2022.0: 2082.531914893618,
 2023.0: 2105.491489361703,
 2024.0: 2128.4393617021274,
 2025.0: 2151.3882978723414,
 2026.0: 2174.3436170212776,
 2027.0: 2197.3021276595746,
 2028.0: 2220.245744680851,
 2029.0: 2243.205319148937,
 2030.0: 2266.153191489363}